In [ ]:
!git clone https://github.com/ThibaultGROUEIX/AtlasNet.git

Cloning into 'AtlasNet'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 449 (delta 3), reused 10 (delta 3), pack-reused 439
Receiving objects: 100% (449/449), 4.33 MiB | 25.92 MiB/s, done.
Resolving deltas: 100% (261/261), done.


In [1]:
cd AtlasNet

[Errno 2] No such file or directory: 'AtlasNet'
/home/ec2-user/SageMaker/AtlasNet


In [3]:
!pip install pandas visdom

    100% |████████████████████████████████| 1.4MB 21.7MB/s ta 0:00:01
  Running setup.py bdist_wheel for visdom ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ee/87/ce/a5023722374ca73b57fc8d4284ba6f973c01219b3c385a07e0
  Running setup.py bdist_wheel for torchfile ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile
fastai 1.0.40 requires dataclasses, which is not installed.
fastai 1.0.40 requires nvidia-ml-py3, which is not installed.
thinc 6.12.1 has requirement msgpack<0.6.0,>=0.5.6, but you'll have msgpack 0.6.0 which is incompatible.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
cd trained_models

/home/ec2-user/SageMaker/AtlasNet/trained_models


In [5]:
!wget https://cloud.enpc.fr/s/c27Df7fRNXW2uG3/download --no-check-certificate

--2019-03-14 12:16:32--  https://cloud.enpc.fr/s/c27Df7fRNXW2uG3/download
Resolving cloud.enpc.fr (cloud.enpc.fr)... 195.221.193.80
Connecting to cloud.enpc.fr (cloud.enpc.fr)|195.221.193.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 512136096 (488M) [application/zip]
Saving to: ‘download’

download            100%[===================>] 488.41M  6.77MB/s    in 2m 10s  

2019-03-14 12:18:44 (3.74 MB/s) - ‘download’ saved [512136096/512136096]



In [6]:
!unzip download

Archive:  download
  inflating: ae_atlasnet_25.pth      
  inflating: ae_atlasnet_sphere.pth  
  inflating: ae_baseline.pth         
  inflating: svr_atlas_25.pth        
  inflating: svr_atlas_sphere.pth    
  inflating: svr_baseline.pth        


In [28]:
!ls

toupload.png


In [29]:
%cd ..


/home/ec2-user/SageMaker/AtlasNet


In [31]:
!python inference/demo.py --cuda 0 --input data/plane_input_demo.png

previous weight loaded
Done demoing! Check out results in data/


In [ ]:
#JPG to PNG
from PIL import Image
import numpy as np

threshold=100
dist=5
ip='chair.jpg'
img=Image.open(ip).convert('RGBA')
# np.asarray(img) is read only. Wrap it in np.array to make it modifiable.
arr=np.array(np.asarray(img))
r,g,b,a=np.rollaxis(arr,axis=-1)    
mask=((r>threshold)
      & (g>threshold)
      & (b>threshold)
      & (np.abs(r-g)<dist)
      & (np.abs(r-b)<dist)
      & (np.abs(g-b)<dist)
      )
arr[mask,3]=0
img=Image.fromarray(arr,mode='RGBA')
img.save('chair_out.png')

In [27]:
import sagemaker
from sagemaker import get_execution_role
import os
import urllib.request
import boto3

def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)
        

def upload_to_s3(channel, file):
    s3 = boto3.resource('s3')
    data = open(file, "rb")
    key = channel + '/' + file
    s3.Bucket(bucket).put_object(Key=key, Body=data)
    
def download_from_s3(file):
    s3 = boto3.resource('s3') 
    key= file
    s3.Bucket(bucket).download_file(Key=key, Filename='/atlasnet/'+key) 



role = get_execution_role()
print(role)

sess = sagemaker.Session()
bucket = 'atlasnet'
prefix = 'atlasnet/output'
item = 'plane_input_demo.png'
input_image_location = 's3://{}/{}'.format(bucket, item)
#download('https://s3.us-east-2.amazonaws.com/atlasnet/input.png')
#upload_to_s3('output', 'toupload.png')

arn:aws:iam::108546266123:role/service-role/AmazonSageMaker-ExecutionRole-20190314T173791


In [2]:
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer

class JSONPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, json_serializer, json_deserializer)




In [5]:
import sagemaker
role = sagemaker.get_execution_role()
from sagemaker.pytorch import PyTorchModel
trained_model_location="s3://atlasnet/trained_models_tar.tar.gz"
model_1 = PyTorchModel(model_data=trained_model_location,
                    role=
                     framework_version='1.0.0',
                     entry_point='auxiliary/model.py',
                     source_dir='auxiliary',
                     )
# role=role,
#predictor_cls=JSONPredictor


TypeError: __init__() missing 1 required positional argument: 'role'

In [4]:
predictor_1 = model_1.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

INFO:sagemaker:Creating model with name: sagemaker-pytorch-2019-03-30-00-34-03-340
INFO:sagemaker:Creating endpoint with name sagemaker-pytorch-2019-03-30-00-34-03-340


----------------------------------------------------------------------------------------------------------------------------------------------------*

ValueError: Error hosting endpoint sagemaker-pytorch-2019-03-30-00-34-03-340: Failed Reason:  The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.

In [1]:
!tar -pcvzf trained_models_tar.tar.gz model_tar_gz

model_tar_gz/
model_tar_gz/svr_atlas_25.pth
model_tar_gz/.ipynb_checkpoints/


In [ ]:
!ls

In [3]:
#2 Attempt from fast.ai tutorial
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer
class ImagePredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(endpoint_name, sagemaker_session=sagemaker_session, serializer=None, 
                         deserializer=json_deserializer, content_type='image/png')

In [4]:
import sagemaker
role = sagemaker.get_execution_role()
from sagemaker.pytorch import PyTorchModel
trained_model_location="s3://atlasnet/trained_models_tar.tar.gz"
model = PyTorchModel(model_data=trained_model_location,
                     role=role,
                     framework_version='1.0.0',
                     entry_point='serve.py', 
                     predictor_cls=ImagePredictor)


In [6]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-pytorch-2019-03-29-14-10-08-647
INFO:sagemaker:Creating endpoint with name sagemaker-pytorch-2019-03-29-14-10-08-647


-------------------------------------------------------------------------------------------------------------------------------------*

ValueError: Error hosting endpoint sagemaker-pytorch-2019-03-29-14-10-08-647: Failed Reason:  The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.

In [7]:
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::108546266123:role/service-role/AmazonSageMaker-ExecutionRole-20190314T173791
